In [10]:
#loading time periods
import pyodbc 
from datetime import datetime
from datetime import timezone
from tqdm.notebook import tqdm
import pandas as pd
import requests

repo_owner='kubernetes'
server = 'localhost' 
database = 'kubernetes' 
username = 'sa' 
password = 'test' 
repo_folder = f"C:\\Users\\f2kazemi\\Desktop\\cs848\\Real_Reviewer Recommender\\{database}\\{database}"
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [11]:
data = pd.read_csv(f'./data/{database}.csv',dtype={'fix':'str','fixes':'str','fileschanged':'str'})  

In [12]:
commits_data = {}
for i in tqdm(range(len(data))):
    commits_data[data.iloc[i]['commit_hash']] = data.iloc[i][['commit_hash','ns','nd','nf','entropy','la','ld','lt','ndev','age','nuc'
                   ,'exp','rexp','sexp','author_date_unix_timestamp','author_date','contains_bug','fileschanged']]


In [13]:
# fetch all the PRs
cursor.execute(f"""SELECT [Id],[Number],[PRCommits],[Metrics] from [{database}].[dbo].[PullRequests]""")
PRs = cursor.fetchall() 


In [14]:
def caluclatePathParams(files):
    subsystems  = []
    directories = []
    
    for file in files:
        pathSlices = file.split('/')
        if len(pathSlices)==1:
            subsystems.append('root')
            directories.append('/')
        else:
            subsystems.append(pathSlices[0])
            directories.append('/'.join(pathSlices[:-1]))
    ns = len(list(set(subsystems)))
    nd = len(list(set(directories)))
    nf = len(list(set(files)))
    return ns, nd, nf


In [15]:
import os
import sys
import re

def get_detail_change_lines(diff):
    re_start_line = re.compile("(@@ -(\d+),(\d+) +\+)(\d+)(,(\d)+ @@)")
    start_line = 0
    change_lines = []
    chang_dict = {}
    additions = 0
    deletions = 0
    for line in diff:
        #print line
        if line.startswith("@@"):
            re_start_line_g = re_start_line.match(line)
            if re_start_line_g:
                start_line = int(re_start_line_g.group(2))-1
                #print start_line
            else:
                pass
                #print line
        elif line.startswith("+") and start_line > 0:
            additions += 1
            start_line = start_line + 1
            change_lines.append(start_line)
        elif line.startswith("-"):
            deletions += 1
            pass
        elif start_line > 0:
            start_line = start_line + 1
        else:
            pass
   # print change_lines
    return {'changed_lines':change_lines, 'additions':additions,'deletions':deletions,'modifications':additions+deletions}



def get_file_name(line):
    com = re.compile(r"(diff --git a)(.*)( b/)")
    m = com.match(line)
    file_name = ""
    if m:
        file_name = m.group(2)
    return file_name




In [16]:
import math
def calculateEntropy(fileChanges,totalModification):
    entropy=0
    if totalModification==0:
        return 0
    for file in fileChanges:
        avg = fileChanges[file]['modifications']/totalModification
        if avg > 0:
            entropy -=  avg * math.log(avg,2)
    return entropy

In [19]:
import json
# re-fetch all the PRs
cursor.execute(f"""SELECT [Id],[Number],[PRCommits],[Metrics] from [{database}].[dbo].[PullRequests] where Metrics is NULL""")
PRs = cursor.fetchall() 

PRMetrics = {}
# for PR in tqdm(PRs[:10]):
i=0
maxLine = 1000000;
while i < len(PRs):
    PR = PRs[i]
    if PR[3] and len(PR[3]):
        print("going next!")
        i +=1
        continue
    print(f"{i}=>progress:{i/len(PRs)*100}, PRID: {PR[0]}")
    try:
        PRcommitInfo =[]
        if PR[2] is None or len(PR[2]) == 0:
            i +=1
            continue;

        commitShaList = PR[2].split(",")

        # We get its commits info
        for commit in commitShaList:
            if commit in commits_data:
                PRcommitInfo.append(commits_data[commit])
        
        lt   = 0
        age  = 0
        ndev = 0
        exp  = 0
        rexp = 0
        sexp = 0
        nuc  = 0
        
        total_files_no = 0
        allFiles = []
        files = []
        for commitInfo in PRcommitInfo:

            if type(commitInfo['fileschanged'])==str:
                files = commitInfo['fileschanged'][:-14].split(",CAS_DELIMITER,");
            else:
                files = []
            allFiles += files
            total_files_no += len(files)
            age  += commitInfo['age'] * len(files)
            lt   += commitInfo['lt'] * len(files)
            ndev += commitInfo['ndev']
            exp  += commitInfo['exp']
            rexp += commitInfo['rexp']
            sexp += commitInfo['sexp']
            nuc  += commitInfo['nuc']
            
        if len(PRcommitInfo):
            ndev  /= len(PRcommitInfo)
            exp   /= len(PRcommitInfo)
            rexp  /= len(PRcommitInfo)
            sexp  /= len(PRcommitInfo)
            nuc   /= len(PRcommitInfo)
        else:
            ndev = 0
            exp  = 0
            rexp = 0
            sexp = 0
            nuc  = 0
            
        if total_files_no!=0:
            lt /= total_files_no
            age /= total_files_no
        else:
            lt = 0
            age= 0
            
        ns, nd, nf = caluclatePathParams(files)
        
        # Get PR parameters
        response = requests.get(f'https://api.github.com/repos/{repo_owner}/{database}/pulls/{PR[1]}',auth=('githubUser','githubToken'));
        info = response.json()
        la  = info['additions']
        ld  = info['additions']
        nf2 = info['changed_files']
        
        # Get the diff
        res_dict = {}
        response = requests.get(f'https://patch-diff.githubusercontent.com/raw/{repo_owner}/{database}/pull/{PR[1]}.diff',auth=('githubUser','githubToken'));
        diff = response.text#open('diff.patch')
        lines = diff.split("\n") 
        line = lines.pop(0)
        if line and len(lines) < maxLine:
            while line:
                if line.startswith("diff --git"):
                    file_name = get_file_name(line)
                    #print file_name
                    detail_change = []
                    while line:
                        line = lines.pop(0)
                        if(line.startswith("diff --git")):
                            res_dict[file_name] = get_detail_change_lines(detail_change)
                            break
                        detail_change.append(line)
                    if not line:
                        res_dict[file_name] = get_detail_change_lines(detail_change)
                else:
                    line = lines.pop(0)
        entropy = calculateEntropy(res_dict,(ld+la))
        
        # Now we have all the parameters in place, just save them to the database
        metrics = {
            'lt':lt,
            'age':age,
            'ndev':ndev,
            'exp':exp,
            'rexp':rexp,
            'sexp':sexp,
            'ns':ns,
            'nd':nd,
            'nf':nf2,
            'nuc':nuc,
            'la':la,
            'ld':ld,
            'entropy':entropy,
            'diff':res_dict,
        }
        
        i +=1


        cursor.execute(f"""UPDATE [{database}].[dbo].[PullRequests]
            SET [Metrics]='{json.dumps(metrics)}'
            WHERE id='{PR[0]}'
        """)
        cnxn.commit()
    except Exception as e: 
        print('Error occured!',PR,e)
#         if info['message'].find('API rate')!= -1:
        response = requests.get(f'https://api.github.com/rate_limit',auth=('githubUser','githubToken'));
        res = response.json()
        while res['resources']['core']['remaining']==0:
            sleepingBeauty = 120
            time.sleep(sleepingBeauty)
            response = requests.get(f'https://api.github.com/rate_limit',auth=('githubUser','githubToken'));
            res = response.json()

0=>progress:0.0, PRID: 22188434
1=>progress:1.0101010101010102, PRID: 27037221
2=>progress:2.0202020202020203, PRID: 27525551
3=>progress:3.0303030303030303, PRID: 28582275
4=>progress:4.040404040404041, PRID: 31157667
5=>progress:5.05050505050505, PRID: 31741653
6=>progress:6.0606060606060606, PRID: 31923096
7=>progress:7.07070707070707, PRID: 32395044
8=>progress:8.080808080808081, PRID: 33698394
9=>progress:9.090909090909092, PRID: 34964259
10=>progress:10.1010101010101, PRID: 36246307
11=>progress:11.11111111111111, PRID: 36874696
12=>progress:12.121212121212121, PRID: 37198114
13=>progress:13.131313131313133, PRID: 38853056
14=>progress:14.14141414141414, PRID: 39300367
15=>progress:15.151515151515152, PRID: 40254504
16=>progress:16.161616161616163, PRID: 42003479
17=>progress:17.17171717171717, PRID: 42260662
18=>progress:18.181818181818183, PRID: 42428075
19=>progress:19.19191919191919, PRID: 45616709
20=>progress:20.2020202020202, PRID: 46028643
21=>progress:21.21212121212121, 